In [15]:
import pickle
import pandas as pd
from sklearn.feature_extraction import DictVectorizer
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import root_mean_squared_error
from sklearn.pipeline import make_pipeline
from mlflow.tracking import MlflowClient
import mlflow

In [16]:
mlflow.set_tracking_uri("http://127.0.0.1:5000")
mlflow.set_experiment("green-taxi-duration")

<Experiment: artifact_location='s3://yezer-artifacts-remote-01/2', creation_time=1750755959246, experiment_id='2', last_update_time=1750755959246, lifecycle_stage='active', name='green-taxi-duration', tags={}>

In [17]:
def read_dataframe(filename: str):
    df = pd.read_parquet(filename)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.dt.total_seconds() / 60
    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    return df


def prepare_dictionaries(df: pd.DataFrame):
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
    categorical = ['PU_DO']
    numerical = ['trip_distance']
    dicts = df[categorical + numerical].to_dict(orient='records')
    return dicts

In [18]:
df_train = read_dataframe('/home/yezer/projects/mlops-zoomcamp/01-intro/data/green_tripdata_2021-01.parquet')
df_val = read_dataframe('/home/yezer/projects/mlops-zoomcamp/01-intro/data/green_tripdata_2021-02.parquet')
df_val2 = read_dataframe('/home/yezer/projects/mlops-zoomcamp/01-intro/data/green_tripdata_2021-03.parquet')

target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values
y_val2 = df_val2[target].values

dict_train = prepare_dictionaries(df_train)
dict_val = prepare_dictionaries(df_val)
dict_val2 = prepare_dictionaries(df_val2)


In [19]:
with mlflow.start_run():
    params = dict(max_depth=20, n_estimators=100, min_samples_leaf=10, random_state=0)
    mlflow.log_params(params)

    # Option 1: Save pipeline (vectorizer + model together)
    pipeline = make_pipeline(
        DictVectorizer(),
        RandomForestRegressor(**params, n_jobs=-1)
    )

    pipeline.fit(dict_train, y_train)
    y_pred = pipeline.predict(dict_val)

    rmse = root_mean_squared_error(y_pred, y_val)
    print(params, rmse)
    mlflow.log_metric('rmse', rmse)

    # Log the entire pipeline as a model
    # This saves both the DictVectorizer and RandomForestRegressor as one unit
    # Advantage: Everything is packaged together, easier to load and use
    # Disadvantage: Less flexibility if you need to access components separately
    mlflow.sklearn.log_model(pipeline, artifact_path="model")
    
    # Option 2: Save vectorizer and model separately (commented out)
    # dv = DictVectorizer()
    # X_train = dv.fit_transform(dict_train)
    # X_val = dv.transform(dict_val)
    # model = RandomForestRegressor(**params, n_jobs=-1)
    # model.fit(X_train, y_train)
    # 
    # # Save vectorizer separately
    # with open("dict_vectorizer.bin", "wb") as f_out:
    #     pickle.dump(dv, f_out)
    # mlflow.log_artifact("dict_vectorizer.bin")
    # mlflow.sklearn.log_model(model, artifact_path="model")
    #
    # Advantage: Can access vectorizer and model independently
    # Disadvantage: More complex to manage two separate artifacts
    
    # Get the current run ID for downloading the model later
    run_id = mlflow.active_run().info.run_id
    print(f"Model logged with run_id: {run_id}")

{'max_depth': 20, 'n_estimators': 100, 'min_samples_leaf': 10, 'random_state': 0} 6.7558229919200725


2025/06/24 14:53:14 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/06/24 14:53:27 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.
2025/06/24 14:53:27 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Model logged with run_id: 008260e4b5964963ac9d0b969c887af2
🏃 View run spiffy-turtle-112 at: http://127.0.0.1:5000/#/experiments/2/runs/008260e4b5964963ac9d0b969c887af2
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/2


In [12]:
MLFLOW_TRACKING_URI = "http://127.0.0.1:5000"
RUN_ID = 'c06055301c0b4bec9f7fff83866d7f21'

# Download the entire pipeline model
logged_model = f'runs:/{RUN_ID}/model'

# Load the model using mlflow
loaded_model = mlflow.sklearn.load_model(logged_model)

# Alternatively, you can download the model artifacts directly
# path = client.download_artifacts(run_id=run_id, path='model')
# loaded_model = mlflow.sklearn.load_model(path)

print(f"Model loaded from run_id: {RUN_ID}")
print(f"Model type: {type(loaded_model)}")

y_pred = loaded_model.predict(dict_val)
rmse = root_mean_squared_error(y_pred, y_val)
print(rmse)

y_pred = loaded_model.predict(dict_val2)
rmse = root_mean_squared_error(y_pred, y_val2)
print(rmse)


Model loaded from run_id: c06055301c0b4bec9f7fff83866d7f21
Model type: <class 'sklearn.pipeline.Pipeline'>
6.7558229919200725
6.674615873838019


In [12]:
client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

In [14]:
path = client.download_artifacts(run_id=RUN_ID, path='dict_vectorizer.bin')
print(path)

None


In [22]:
with open(path, 'rb') as f_out:
    dv = pickle.load(f_out)

In [23]:
dv

DictVectorizer()